In [1]:
import pandas as pd
import numpy as np

from simulator import *

from IPython.display import display

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
def get_dataset1():
    dataset = []
    for i in [1,2,3]:
        df = pd.read_csv(f'../data/table{i}.csv', dtype={'Genotype': str}).set_index('Genotype').T
        df = df.rename_axis('Label')
        dataset.append(df)
    return dataset
dataset1 = get_dataset1()
#for table in dataset1:
#    display(table)

In [3]:
def get_dataset2():
    df = pd.read_csv('../data/barlow.csv', index_col=0)
    df = df.sort_index(axis=1)
    df = df.rename_axis('Genotype', axis='columns')
    return df
dataset2 = get_dataset2()
#dataset2

In [104]:
def genotype_strings(num):
    return(["{0:0>4b}".format(n) for n in range(num)])

def plot_simulation(results, genotypes_to_plot=[]):
    trace = results['trace']
    if not genotypes_to_plot:
        genotypes_to_plot = genotype_strings(len(trace))
    timesteps = len(next(iter(trace.values())))
    #           length of an arbitrary entry from dictionary
    index = np.array(range(0, timesteps), np.int64)
    
    # data
    data = []
    for g in genotypes_to_plot:
        data.append(go.Scatter(
            x=index,
            y=trace[g],
            name=g,
            line=dict(width=1)
        ))

    # vertical lines
    for crit in ['T_1', 'T_d', 'T_f']:
        data.append(go.Scatter(
            x=[results[crit], results[crit]],
            y=[0, 10**10],
            line=dict(color='black', width=1),
            name = crit,
            mode = 'lines',
            showlegend = False
        ))

    # vertical line annotations
    vlines = []
    for crit, text in zip(['T_1', 'T_d', 'T_f'], ['<i>T</i><sub>1</sub>', '<i>T</i><sub>d</sub>', '<i>T</i><sub>f</sub>']):
        vlines.append(dict(
            x=results[crit],
            y=9.75,
            xref='x',
            yref='y',
            text=text,
            showarrow=False,
            xanchor='left'
        ))
        
    # layout
    layout = go.Layout(
        yaxis=dict(
            type='log',
            range=[0,10],
            exponentformat='power'
        ),
        width=600,
        height=400,
        margin=dict(t=50, b=50),
        annotations=vlines
    )
        
    fig = go.Figure(data=data, layout=layout)
    
    iplot(fig, show_link=False)

In [105]:
landscape = dataset1[2].loc['53.60μM'].tolist()
results = simulate(landscape, timesteps=2000)
plot_simulation(results, ["0000", "0010", "0110", "1110"])